<a href="https://colab.research.google.com/github/olcaykursun/ML/blob/main/neuralnets/autoencoder_answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np

# Load the Iris dataset
iris = load_iris()
x_data = iris.data[:,:3]             #I wanted to make it more challenging by using 3 of the features instead of 4.
y_data = to_categorical(iris.target)

# Split the dataset into training and testing
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, train_size=15, random_state=42, stratify=iris.target)

# Create a standalone classifier
classifier = models.Sequential([
    layers.Input(shape=(3,)),
    layers.Dense(50, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(3, activation='softmax')
])

# Compile the classifier
classifier.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

# Train the classifier
classifier.fit(x_train, y_train, epochs=300, batch_size=5, verbose=0)

# Evaluate the classifier
test_loss, test_acc = classifier.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

2024-11-18 10:39:40.530388: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-11-18 10:39:40.530430: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-11-18 10:39:40.530441: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-11-18 10:39:40.530483: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-18 10:39:40.530505: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-11-18 10:39:40.801769: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 0s 12ms/step - loss: 0.2231 - accuracy: 0.9481
Test accuracy: 0.9481


In [ ]:
# Create an autoencoder
input_layer = Input(shape=(3,))
encoded = layers.Dense(50, activation='relu')(input_layer)
encoded = layers.Dense(10, activation='relu')(encoded)
decoded = layers.Dense(50, activation='sigmoid')(encoded)
decoded = layers.Dense(3, activation='linear')(decoded)

autoencoder = models.Model(inputs=input_layer, outputs=decoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
autoencoder.fit(x_data, x_data, epochs=300, verbose=0)

# Create a classifier using the encoder part of the autoencoder
encoder = models.Model(inputs=input_layer, outputs=encoded)
encoder.trainable = False

# Add a softmax layer for classification
encoded_input = encoder(input_layer)
classifier_output = layers.Dense(3, activation='softmax')(encoded_input)
classifier = models.Model(inputs=input_layer, outputs=classifier_output)

# Compile the classifier
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Warmup the softmax
classifier.fit(x_train, y_train, epochs=300, batch_size=5, verbose=0)

# Evaluate the classifier
test_loss, test_acc = classifier.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

5/5 [==============================] - 0s 11ms/step - loss: 0.4225 - accuracy: 0.9037
Test accuracy: 0.9037


In [ ]:
# Unfreeze and recompile the classifier
encoder.trainable = True
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Finetune
classifier.fit(x_train, y_train, epochs=200, batch_size=5, verbose=0)

# Evaluate the classifier
test_loss, test_acc = classifier.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

#Finally we see the benefit of autoencoder
#The benefit will be more on more complex problems (that statement maybe tested on MNIST for example)

5/5 [==============================] - 0s 10ms/step - loss: 0.1478 - accuracy: 0.9556
Test accuracy: 0.9556
